<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
import pandas as pd
import io
from js import fetch

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)


In [7]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [17]:
# Task 1: Mark all launch sites on a map
site_lat = spacex_df['Lat'].mean()
site_lon = spacex_df['Long'].mean()
spacex_map = folium.Map(location=[site_lat, site_lon], zoom_start=4)
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(spacex_map)

spacex_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [18]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [19]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [20]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [21]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [22]:
import folium
from folium.features import DivIcon

# Initialize map centered at NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circles and markers for each site
for _, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Circle
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site_name)).add_to(site_map)
    
    # Marker with label
    folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [23]:
# Task 2: Mark the success/failed launches for each site on the map
import folium

# Initialize map centered at NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add markers for each launch attempt
for _, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    outcome = row['class']  # 1 = success, 0 = failure
    
    # Color code: green for success, red for failure
    color = 'green' if outcome == 1 else 'red'
    
    folium.Marker(
        location=coordinate,
        popup=f"{site_name} - {'Success' if outcome == 1 else 'Failure'}",
        icon=folium.Icon(color=color, icon='rocket')
    ).add_to(site_map)

site_map


In [24]:
# Explaining Finding
## Are all launch sites in proximity to the Equator line?
  # > SpaceX launch sites are located in Florida (~28°N), Texas (~26°N), and California (~34°N), which are far north of the Equator at 0°. While not equatorial, these southern U.S. 
  # > locations are chosen to take advantage of Earth’s rotational speed, giving rockets an extra velocity boost when launching eastward. 
  # > This positioning improves efficiency and reduces fuel needs compared to sites further north.
## Are all launch sites in very close proximity to the coast?
  # > All SpaceX launch sites are situated along coastlines — Cape Canaveral and Kennedy Space Center on the Atlantic, Vandenberg on the Pacific, and Boca Chica on the Gulf of Mexico. 
  # > Coastal locations are deliberate, allowing rocket stages and debris to fall safely into the ocean while providing clear flight paths over water. This ensures both safety for nearby populations and optimal launch trajectories.


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [25]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [27]:
from folium.plugins import MarkerCluster

# Initialize map centered at NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch attempt
for _, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    outcome = row['class']  # 1 = success, 0 = failure
    
    # Color code: green for success, red for failure
    color = 'green' if outcome == 1 else 'red'
    
    folium.Marker(
        location=coordinate,
        popup=f"{site_name} - {'Success' if outcome == 1 else 'Failure'}",
        icon=folium.Icon(color=color, icon='rocket')
    ).add_to(marker_cluster)

# Display the map
site_map



*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [28]:
# Create a new column 'marker_color' based on the 'class' value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Preview the updated DataFrame
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [30]:
# Initialize map centered at NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object and add to map
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each launch record
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    site_name = record['Launch Site']
    outcome = 'Success' if record['class'] == 1 else 'Failure'
    
    # Create marker with customized icon
    marker = folium.Marker(
        location=coordinate,
        popup=f"{site_name} - {outcome}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket')
    )
    
    # Add marker to cluster
    marker_cluster.add_child(marker)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [34]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math

# Haversine formula
def haversine(coord1, coord2):
    lat1, lon1, lat2, lon2 = map(math.radians, [*coord1, *coord2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    return 6371 * 2 * math.asin(math.sqrt(a))  # Earth radius in km

# Proximities of interest
proximities = {
    "Orlando": (28.538336, -81.379234),
    "Los Angeles": (34.052235, -118.243683),
    "Brownsville": (25.901747, -97.497483)
}

# One compact loop
for _, row in launch_sites_df.iterrows():
    site = (row['Lat'], row['Long'])
    print(f"\n{row['Launch Site']}:")
    print("\n".join(
        [f"  Distance to {city}: {haversine(site, coord):.2f} km"
         for city, coord in proximities.items()]
    ))



CCAFS LC-40:
  Distance to Orlando: 78.37 km
  Distance to Los Angeles: 3609.59 km
  Distance to Brownsville: 1697.32 km

CCAFS SLC-40:
  Distance to Orlando: 78.42 km
  Distance to Los Angeles: 3609.61 km
  Distance to Brownsville: 1697.39 km

KSC LC-39A:
  Distance to Orlando: 71.63 km
  Distance to Los Angeles: 3602.78 km
  Distance to Brownsville: 1690.70 km

VAFB SLC-4E:
  Distance to Orlando: 3753.42 km
  Distance to Los Angeles: 226.70 km
  Distance to Brownsville: 2415.21 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [35]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [37]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
# Kennedy Space Center (LC-39A)
lat1, lon1 = 28.573255, -80.646895
# Orlando
lat2, lon2 = 28.538336, -81.379234

distance_km = calculate_distance(lat1, lon1, lat2, lon2)
print(f"Distance from LC-39A to Orlando: {distance_km:.2f} km")


Distance from LC-39A to Orlando: 71.65 km


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [39]:
import folium 
from folium.features import DivIcon 
# Launch site coordinates (LC-39A) 
launch_site_lat, launch_site_lon = 28.573255, -80.646895 
# Closest coastline coordinates (from MousePosition) 
coastline_lat, coastline_lon = 28.56367, -80.57163 
# Calculate distance using your function 
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, 
                                        coastline_lat, coastline_lon) 
# Initialize map centered at launch site 
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)


In [40]:
# Add marker for launch site 
folium.Marker( [launch_site_lat, launch_site_lon], 
              popup="Launch Site: LC-39A", 
              icon=folium.Icon(color="blue", icon="rocket") 
             ).add_to(site_map) 
# Add marker for coastline point 
folium.Marker( [coastline_lat, coastline_lon], 
              popup="Closest Coastline", 
              icon=folium.Icon(color="green", icon="flag") 
             ).add_to(site_map) 
# Add distance marker (text label) 
distance_marker = folium.Marker( 
    [coastline_lat, coastline_lon], 
    icon=DivIcon( 
        icon_size=(20,20), 
        icon_anchor=(0,0), 
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % 
             "{:10.2f} KM".format(distance_coastline), 
    ) 
) 
distance_marker.add_to(site_map) 
# Display map 
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [41]:
import folium

# Launch site coordinates (LC-39A)
launch_site_lat, launch_site_lon = 28.573255, -80.646895

# Closest coastline coordinates (from MousePosition)
coastline_lat, coastline_lon = 28.56367, -80.57163

# Initialize map centered at launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add markers for clarity
folium.Marker([launch_site_lat, launch_site_lon], popup="Launch Site").add_to(site_map)
folium.Marker([coastline_lat, coastline_lon], popup="Coastline").add_to(site_map)

# Create PolyLine between launch site and coastline
coordinates = [(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color="red")

# Add line to map
site_map.add_child(lines)

# Display map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [49]:
import folium
from folium.features import DivIcon

# Launch site coordinates (LC-39A)
launch_site_lat, launch_site_lon = 28.573255, -80.646895

# Example railway coordinates (from MousePosition hover)
railway_lat, railway_lon = 28.5721, -80.6205   # replace with actual railway coords

# Calculate distance
distance_railway = calculate_distance(launch_site_lat, launch_site_lon,
                                      railway_lat, railway_lon)

# Initialize map
railway_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Launch site marker
folium.Marker([launch_site_lat, launch_site_lon],
              popup="Launch Site",
              icon=folium.Icon(color="blue", icon="rocket")).add_to(railway_map)

# Railway marker with custom railway picture
railway_icon = folium.CustomIcon(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png",
    icon_size=(30, 30)
)
folium.Marker([railway_lat, railway_lon],
              popup="Railway",
              icon=railway_icon).add_to(railway_map)

# Distance label marker
folium.Marker([railway_lat, railway_lon],
              icon=DivIcon(
                  icon_size=(20,20),
                  icon_anchor=(0,0),
                  html=f'<div style="font-size:12px; color:#d35400;"><b>{distance_railway:.2f} KM</b></div>'
              )
).add_to(railway_map)

# PolyLine between launch site and railway
folium.PolyLine(
    [(launch_site_lat, launch_site_lon), (railway_lat, railway_lon)],
    weight=2, color="red"
).add_to(railway_map)

# Display map
railway_map


In [50]:
import folium
from folium.features import DivIcon

# Launch site coordinates (LC-39A)
launch_site_lat, launch_site_lon = 28.573255, -80.646895

# Example highway coordinates (from MousePosition hover)
highway_lat, highway_lon = 28.5650, -80.6000   # replace with actual highway coords

# Calculate distance
distance_highway = calculate_distance(launch_site_lat, launch_site_lon,
                                      highway_lat, highway_lon)

# Initialize map
highway_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Launch site marker
folium.Marker([launch_site_lat, launch_site_lon],
              popup="Launch Site",
              icon=folium.Icon(color="blue", icon="rocket")).add_to(highway_map)

# Highway marker with custom icon
highway_icon = folium.CustomIcon(
    "https://cdn-icons-png.flaticon.com/512/854/854894.png",  # example highway icon
    icon_size=(30, 30)
)
folium.Marker([highway_lat, highway_lon],
              popup="Highway",
              icon=highway_icon).add_to(highway_map)

# Distance label marker
folium.Marker([highway_lat, highway_lon],
              icon=DivIcon(
                  icon_size=(20,20),
                  icon_anchor=(0,0),
                  html=f'<div style="font-size:12px; color:#d35400;"><b>{distance_highway:.2f} KM</b></div>'
              )
).add_to(highway_map)

# PolyLine between launch site and highway
folium.PolyLine(
    [(launch_site_lat, launch_site_lon), (highway_lat, highway_lon)],
    weight=2, color="red"
).add_to(highway_map)

# Display map
highway_map


In [51]:
# Launch site coordinates (LC-39A)
launch_site_lat, launch_site_lon = 28.573255, -80.646895

# Example city coordinates (Orlando)
city_lat, city_lon = 28.538336, -81.379234   # replace with actual city coords if needed

# Calculate distance
distance_city = calculate_distance(launch_site_lat, launch_site_lon,
                                   city_lat, city_lon)

# Initialize map
city_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=8)

# Launch site marker
folium.Marker([launch_site_lat, launch_site_lon],
              popup="Launch Site",
              icon=folium.Icon(color="blue", icon="rocket")).add_to(city_map)

# City marker with custom icon
city_icon = folium.CustomIcon(
    "https://cdn-icons-png.flaticon.com/512/684/684908.png",  # example city/building icon
    icon_size=(30, 30)
)
folium.Marker([city_lat, city_lon],
              popup="City: Orlando",
              icon=city_icon).add_to(city_map)

# Distance label marker
folium.Marker([city_lat, city_lon],
              icon=DivIcon(
                  icon_size=(20,20),
                  icon_anchor=(0,0),
                  html=f'<div style="font-size:12px; color:#d35400;"><b>{distance_city:.2f} KM</b></div>'
              )
).add_to(city_map)

# PolyLine between launch site and city
folium.PolyLine(
    [(launch_site_lat, launch_site_lon), (city_lat, city_lon)],
    weight=2, color="red"
).add_to(city_map)

# Display map
city_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [53]:
## Railways
 # > Observation: From your plotted railway markers and connecting lines, launch sites (like LC‑39A at Kennedy Space Center) are indeed close to railways.
 # > Reasoning: Railways are often built near large industrial complexes to transport heavy equipment. Rockets and launch hardware are massive, so proximity to rail lines makes logistical sense. NASA historically used rail to move Saturn V and Shuttle components.
## Highways
 # > Observation: Launch sites are also close to highways.
 # > Reasoning: Highways provide access for workers, tourists, and supply chains. Even though rockets themselves aren’t transported by road, support equipment, fuel, and personnel need reliable road access. So highways are deliberately routed nearby.
## Coastline
 # > Observation: Launch sites are very close to coastlines — your distance calculations show only a few kilometers.
 # > Reasoning: This is intentional. Launching near the ocean ensures that rockets fly eastward over water, reducing risk to populated areas if debris falls. It also provides safe zones for booster landings and splashdowns.
## Cities
 # > Observation: Launch sites keep a significant distance away from cities (e.g., Orlando is ~70 km from LC‑39A).
 # > Reasoning: Safety is the main factor. Rockets are hazardous, and launches can fail. Keeping sites away from dense urban centers minimizes risk to people and property. At the same time, they’re close enough to cities to benefit from infrastructure, workforce, and tourism.
### Findings Summary
 # > launch sites are strategically placed to balance logistics, safety, and accessibility. They hug coastlines, connect to transport networks, but avoid being too close to major population centers.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
